Explore some CT scans from UVMMC


## Imports

In [ ]:

from glob import glob
import imageio
from IPython.display import Image
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib
from nibabel.testing import data_path
import numpy as np
import os
import pandas as pd
from pathlib import Path
import projd
import pydicom
import re
import seaborn as sns
import scipy.ndimage
from scipy.ndimage.interpolation import rotate
from skimage import morphology
from skimage import measure
from skimage.transform import resize
import uuid

np.set_printoptions(precision=2, suppress=True)
%matplotlib inline
sns.set()

## Constants

In [ ]:
data_dir = Path('~/data/2018').expanduser()
normal_scans_dir = data_dir / 'uvmmc/nifti_normals'
gif_path = str(Path('~/Downloads/test.gif').expanduser())


In [ ]:
def temp_gif_path():
    return str(Path('~/Downloads').expanduser() / ('tmp_' + uuid.uuid4().hex + '.gif'))

    
def get_nifti_files(path):
    '''
    Return a list of Path objs for every .nii file within path.
    '''
    return list(path.glob('**/*.nii'))

def sample_stack(stack, rows=3, cols=3, start_with=0, show_every=3, r=0):
    '''
    stack: 3-d voxel array.
    '''
    fig, ax = plt.subplots(rows, cols, figsize=[20, 20])
    for i in range(rows * cols):
        ind = start_with + i * show_every
        ax[i // cols, i % cols].set_title('slice %d' % ind)
        
        if r == 0:
            ax[i // cols, i % cols].imshow(stack[:, :, ind], cmap='gray')
        else:
            ax[i // cols, i % cols].imshow(rotate(stack[:, :, ind], r), cmap='gray')
        
        
        ax[i // cols, i % cols].axis('off')
    plt.show()


def make_animated_gif(path, img, start=0, stop=None, step=1):
    '''
    Create animated gif of 3d image, where each frame is a 2-d image taken from 
    iterating across the 3rd dimension.  E.g. the ith 2d image is img[:, :, i]
    path: where to save the animated gif
    img: a 3-d volume
    start: index of 3rd dimension to start iterating at.  default = 0.
    stop: index of 3rd dimension to stop at, not inclusive.  Default is None, meaning stop at img.shape[2].
    step: number of slices to skip    
    '''
    # convert to uint8 to suppress warnings from imageio
    imax = img.max()
    imin = img.min()
    img = 255 * ((img - imin) / (imax - imin)) # scale to 0..255
    img = np.uint8(img)
    
    with imageio.get_writer(path, mode='I') as writer:
        for i in range(start, img.shape[2], step):
            writer.append_data(img[:, :, i])

    
def animate_crop(img, crop=(0, 1, 0, 1, 0, 1), axis=2, step=5):
    '''
    img: a 3d volume to be cropped and animated.
    axis: 0, 1, 2: the axis to animate along.  img will be transposed s.t. this axis is the 3rd axis.
    crop: 6 element list: axis 0 start position, axis 0 end position, axis 1 start position, etc.  Each position 
      is a number in [0.0, 1.0] representing the position as a proportion of that axis.  0.0 is the beginning,
      1.0 the end, and 0.5 the middle.
    step: only include every nth frame in the animation, where each frame is a 2d slice of img.
    return: ipython Image, for display in a notebook.
    '''
    # as a proportion of the total range, range of axis 0, 1, and 2 that should be included in the volume
    prop0 = crop[0:2]
    prop1 = crop[2:4]
    prop2 = crop[4:6]
    # as specific voxel coordinates, range of axis 0, 1, and 2 that should be included in the volume
    pix0 = [int(p * img.shape[0]) for p in prop0]
    pix1 = [int(p * img.shape[1]) for p in prop1]
    pix2 = [int(p * img.shape[2]) for p in prop2]

    cropped_img = img[pix0[0]:pix0[1], pix1[0]:pix1[1], pix2[0]:pix2[1]]
    # rotate axes for animation
    cropped_img = cropped_img.transpose([0,1,2][-(2-axis):] + [0,1,2][:-(2-axis)])
    
    tmp_path = temp_gif_path()
    print('temp gif path:', tmp_path)
    make_animated_gif(tmp_path, cropped_img, step=step)
    return Image(filename=tmp_path)


def animate_scan_info_crop(scan_info, i, crop=(0, 1, 0, 1, 0, 1), axis=0, step=3):
    path = scan_info.loc[i, 'path']
    print('scan path:', path)
    img = nib.load(path).get_data()
    print('scan img shape:', img.shape)
    return animate_crop(img, crop, axis=axis, step=step)
    


In [ ]:
scan_paths = get_nifti_files(normal_scans_dir)

## Load a nifti file

In [ ]:
nft1 = nib.load(str(scan_paths[0]))
print(scan_paths[0])

## Examine the header

The header tells us the voxel size and image dimensions, among other things.

In [ ]:
# This tells us what the file format is, the pixel dimensions, the image dimensions, the affine type.
# This is a nifti 1 file, not a nifti 2 file.
print(nft1.header)

## Examine the Affine

The affine of a nifti file is an affine tranformation matrix relating voxel coordinates to a set of reference coordinates.

The affine type is qform, not sform, as can be seen from the qform_code and sform_code fields of the header.

The reference (world coordinates) are "scanner".

The pixel dimensions are: 0.34  0.34  0.45.  These are mm, I guess.


In [ ]:
# The affine is used to translate between voxels and "world coordinates"
# Unlike other formats, the NIfTI header format can specify this affine in one of three ways:
# the sform affine, the qform affine and the fall-back header affine.

# The affine is a diagonal matrix (+ transpose).  The axis 0 (mediolateral) entry (-0.34) is negative, 
# meaning an increase along the voxel dimension corresonds to a decrease along the world dimensions.
# I think nibabel tries to make voxels RAS+
nft1.affine

## Visualize the Scan

In [ ]:
# get the image data

img1 = nft1.get_data()
img1.shape

In [ ]:
# axial/transverse view of cervical spine (traversing the inferosuperior axis)
# voxel axis 2 == inferosuperior axis, S+ (larger numbers are more superior)
sample_stack(img1, rows=3, cols=3, show_every=20, start_with=200, r=0)


In [ ]:
# frontal/coronal view of cervical spine (traversing the anteroposteior axis)
# voxel axis 1 == anteroposterior axis, A+ (larger numbers are more anterior)
sample_stack(img1.transpose([2, 0, 1]), rows=3, cols=3, show_every=30, start_with=150, r=0)

In [ ]:
# median/sagittal view of cervical spine (traversing the mediolateral axis)
# voxel axis 0 == mediolateral axis, R+ (larger numbers are superior)
sample_stack(img1.transpose([1, 2, 0]), rows=3, cols=3, show_every=20, start_with=img1.shape[0] // 2, r=0)

## What part of the image contains the c2 vertebra?

For img1, the whole c2 verterbra fits within a volume that is ~6% of the size of the total volume.  See the animated gifs below.

### What the c2 (axis) and c1 (atlas) vertebrae look like and how they fit together.

In [ ]:
Image(url='http://www.backpain-guide.com/Chapter_Fig_folders/Ch05_Anatomy_Folder/Ch5_Images/05-3_C1_and_C2.jpg')

### The whole scan, animated

In [ ]:
animate_crop(img1, axis=0, step=15)

In [ ]:
# Crop the total volume to a subspace that contains all of the c2 vertebra.
# as a proportion of the total range, range of axis 0, 1, and 2 that should be included in the volume
prop0 = (0.25, 0.7) # ~45% of total range
prop1 = (0.3, 0.75) # ~45% of total range
prop2 = (.52, 0.80) # ~30% of total range

animate_crop(img1, crop=prop0 + prop1 + prop2, axis=0, step=5)


## Examine distribution of image features across all images

Examine the variance in pixel sizes, in image dimensions.  Are all the affines diagonal matrices (except the transposition part)?  Etc.  This is part of our data quality pipeline, where we learn about the consistencies and variety in the scans. 

In [ ]:
# scan info will contain metadata about each of the scans in the small dataset we are examining
scan_info = pd.DataFrame({'id': [re.sub('\.nii$', '', p.name) for p in scan_paths], 'path': [str(p) for p in scan_paths]})
scan_info.head()

In [ ]:
scan_info['nft'] = scan_info.path.apply(lambda p: nib.load(p))
scan_info['header'] = scan_info.nft.apply(lambda nft: nft.header)
scan_info['affine'] = scan_info.nft.apply(lambda nft: nft.affine)
scan_info['pixdim'] = scan_info.header.apply(lambda h: h['pixdim'][1:4])
scan_info['dim'] = scan_info.header.apply(lambda h: h['dim'][1:4])
scan_info['qform_code'] = scan_info.header.apply(lambda h: h['qform_code'])
scan_info['sform_code'] = scan_info.header.apply(lambda h: h['sform_code'])
scan_info['sizeof_hdr'] = scan_info.header.apply(lambda h: h['sizeof_hdr'])
scan_info['pixdim0'] = scan_info.pixdim.apply(lambda x: x[0])
scan_info['pixdim1'] = scan_info.pixdim.apply(lambda x: x[1])
scan_info['pixdim2'] = scan_info.pixdim.apply(lambda x: x[2])
scan_info['dim0'] = scan_info.dim.apply(lambda x: x[0])
scan_info['dim1'] = scan_info.dim.apply(lambda x: x[1])
scan_info['dim2'] = scan_info.dim.apply(lambda x: x[2])
scan_info['desc'] = scan_info.header.apply(lambda h: h['descrip'])

scan_info.head()
# scan_info.describe()

In [ ]:
# Distributions of pixel dimensions and volume dimensions

fig, ax = plt.subplots(2, 3)# , figsize=[20, 20])
ax[0, 0].hist(scan_info.pixdim0)
ax[0, 1].hist(scan_info.pixdim1)
ax[0, 2].hist(scan_info.pixdim2)
ax[1, 0].hist(scan_info.dim0)
ax[1, 1].hist(scan_info.dim1)
ax[1, 2].hist(scan_info.dim2)
ax[0,0].set_title('pixdim 0')
ax[0,1].set_title('pixdim 1')
ax[0,2].set_title('pixdim 2')
ax[1,0].set_title('dim 0')
ax[1,1].set_title('dim 1')
ax[1,2].set_title('dim 2')
plt.tight_layout()

plt.show()

In [ ]:
# Make some assertions about what assume to be constant across all images

# all images are 512 x 512 x ?
assert (scan_info.dim0 == 512).all()
assert (scan_info.dim1 == 512).all()

# all images use the qform affine
assert (scan_info.qform_code == 1).all()
assert (scan_info.sform_code == 0).all()

# all images have matching pixel sizes in dimensions 0 and 1
assert (scan_info.pixdim0 == scan_info.pixdim1).all()

# all nifti files are nifti 1 format (and so have a header size of 348).
assert (scan_info.sizeof_hdr == 348).all()

In [ ]:
# What are the descriptions
for i in range(len(scan_info)):
    print(scan_info.iloc[i]['id'], scan_info.iloc[i]['desc'])


In [ ]:
# Which scans do not have a vertical pixel dimension that is 0.45 +/- 0.001
scan_info[(scan_info.pixdim2 < 0.449) | (scan_info.pixdim2 > 0.451)]

In [ ]:
# Take a closer look at the variance in mediolateral, anteroposterior, and inferosuperior pixel sizes
fig, ax = plt.subplots(3, figsize=(5, 7))
ax[0].hist(scan_info.pixdim0, bins=100)
ax[0].set_title('Distribution of pixel dimension 0 (mediolateral)')
ax[1].hist(scan_info.pixdim1, bins=100)
ax[1].set_title('Distribution of pixel dimension 1 (anteroposterior)')
ax[2].hist(scan_info.pixdim2, bins=100)
ax[2].set_title('Distribution of pixel dimension 2 (inferosuperior)')
plt.tight_layout()
plt.show()


In [ ]:
print('mode for pixel dimensions 0, 1, 2:', scan_info.pixdim0.mode()[0], scan_info.pixdim1.mode()[0], 
      scan_info.pixdim2.mode()[0])
print('size of mode for pixdim0:', (scan_info.pixdim0 == scan_info.pixdim0.mode()[0]).sum())

In [ ]:
plt.hist(scan_info.dim2, bins=100)
plt.title('Distribution of anterosuperior size of image')
plt.show()

## C2 vertebra locations in the scans

To develop some intuition as to approximately where the c2 vertebra is located in each scan, examine multiple scans and crop the c2 vertebra, like we did earlier for the first scan.

In [ ]:
# axis 0 start, axis 0 end, axis 1 start, axis 1 end, axis 2 start, axis 2 end
# units are fraction of the total axis length, so they are in the range [0.0, 1.0]
c2_crops = [('/Users/tfd/data/2018/uvmmc/nifti_normals/Sept_normals_nii/101944_332.nii',
             (0.25, 0.7, 0.3, 0.75, .52, 0.80)),
           ]


In [ ]:
animate_crop(img1, c2_crops[0][1], step=2, axis=1)

In [ ]:
path = scan_info.loc[1, 'path']
print(path)
img = nib.load(path).get_data()
print(img.shape)


In [ ]:
crop = (0.25, 0.7, 0.3, 0.75, .52, 0.8) # 
animate_crop(img, crop, axis=0, step=3)

In [ ]:
path = scan_info.loc[2, 'path']
print(path)
img = nib.load(path).get_data()
print(img.shape)


In [ ]:
crop = (0.25, 0.7, 0.3, 0.75, .52, 0.8) # 
# crop = (0.1, 0.9, 0.1, 0.9, .3, 0.9) # 
animate_crop(img, crop, axis=0, step=3)

In [ ]:
path = scan_info.loc[3, 'path']
print(path)
img = nib.load(path).get_data()
print(img.shape)


In [ ]:
crop = (0.25, 0.7, 0.3, 0.75, .52, 0.8) # 
# crop = (0.1, 0.9, 0.1, 0.9, .3, 0.9) # 
animate_crop(img, crop, axis=0, step=3)

In [ ]:
animate_scan_info_crop(scan_info, 0, crop=(0.25, 0.7, 0.3, 0.75, .52, 0.8))
# animate_scan_info_crop(scan_info, 0)

In [ ]:
animate_scan_info_crop(scan_info, 1, crop=(.3, .7, .15, .65, .5, .9))

In [ ]:
animate_scan_info_crop(scan_info, 2, crop=(.2, .65, .45, .95, .45, .8))

In [ ]:
animate_scan_info_crop(scan_info, 3, crop=(.35, .75, .2, .65, .5, .8))

In [ ]:
animate_scan_info_crop(scan_info, 4, crop=(.27, .72, .4, .9, .5, .8))

In [ ]:
animate_scan_info_crop(scan_info, 5, crop=(.35, .8, .48, .95, .5, .75))

In [ ]:
animate_scan_info_crop(scan_info, 6, crop=(.28, .72, .3, .72, .5, .8))

In [ ]:
animate_scan_info_crop(scan_info, 7, crop=(.3, .7, .4, .8, .5, .8))

In [ ]:
animate_scan_info_crop(scan_info, 8, crop=(.4, .8, .35, .75, .45, .8))

In [ ]:
animate_scan_info_crop(scan_info, 7, crop=(.5, .9, .3, .8, .45, .8))

## Examine the Variation of Scan Affines

In [ ]:
# Look at all the affine transformations

for i in range(len(scan_info)):
    print(scan_info.iloc[i]['affine'])

In [ ]:
# Check that all the non-translation parts of the affine matrix are diagonal.
# This means that the affine transformation scales/zooms and translates, but does not rotate.

# http://nipy.org/nibabel/coordinate_systems.html#the-affine-matrix-as-a-transformation-between-spaces
def is_diagonal_affine(affine):
    '''
    affine: e.g. 
    array([[  -0.34,    0.  ,    0.  ,   72.45],
           [   0.  ,    0.34,    0.  , -248.76],
           [   0.  ,    0.  ,    0.45,   22.1 ],
           [   0.  ,    0.  ,    0.  ,    1.  ]])
    '''
    mat = affine[:3, :3]
    return np.count_nonzero(mat - np.diag(np.diag(mat))) == 0

# Make some assertions about what we've seen in the affine transformations

# All affines do no rotation (the upper left 3x3 matrix within the affine is diagonal)
assert scan_info['affine'].apply(is_diagonal_affine).all()
# All affines mirror axis 0, but not axis 1 or 2
assert (scan_info['affine'].apply(lambda a: a[0, 0] < 0)).all()
assert (scan_info['affine'].apply(lambda a: a[1, 1] > 0)).all()
assert (scan_info['affine'].apply(lambda a: a[2, 2] > 0)).all()
    